In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_xy,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [2]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [3]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [4]:
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [5]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [6]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [7]:
len(df_ipcc_group)

53258

In [8]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

True

In [9]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [10]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [11]:
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [12]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [13]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [14]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

True

In [15]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')

In [16]:
df_ipcc_ipbes['countries']=df_ipcc_ipbes.apply(lambda row: get_xy('countries',row), axis=1)
df_ipcc_ipbes['year']=df_ipcc_ipbes.apply(lambda row: get_xy('year',row), axis=1)
df_ipcc_ipbes['title']=df_ipcc_ipbes.apply(lambda row: get_xy('title',row), axis=1)
df_ipcc_ipbes['rors']=df_ipcc_ipbes.apply(lambda row: get_xy('rors',row), axis=1)
df_ipcc_ipbes['authors_name']=df_ipcc_ipbes.apply(lambda row: get_xy('authors_name',row), axis=1)
df_ipcc_ipbes['institutions_names']=df_ipcc_ipbes.apply(lambda row: get_xy('institutions_names',row), axis=1)
df_ipcc_ipbes['topics']=df_ipcc_ipbes.apply(lambda row: get_xy('topics',row), axis=1)

In [17]:
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year','title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes','topics']]

In [18]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

True

In [ ]:
update_bso_publications()

In [19]:
bso=pd.DataFrame(get_bso_publications())

In [20]:
df_bso=bso.transpose().reset_index()

In [21]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [22]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [23]:
del df_ipcc_ipbes_bso['rors_y']

In [24]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [25]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes', 'topics', 'bso_local_affiliations'],
      dtype='object')

In [26]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso2_iso3.json')

In [27]:
#df_iso=df_iso.transpose().reset_index().rename(columns={'index':'code',0:'name'})

In [27]:
dict_iso=df_iso.to_dict(orient='records')[0]
dict_iso['XK']='XKX'

In [ ]:
# dict_iso={}
# for i,row in df_iso.iterrows():
#     dict_iso[row['code']]=row['name']
# dict_iso['CW']='Curaçao'
# dict_iso['RS']='Serbia'
# dict_iso['ME']='Montenegro'
# dict_iso['XK']='Kosovo'
# dict_iso['SS']='South Soudan'
# dict_iso['MF']='Saint Martin'

In [28]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [31]:
df_brevet_doi=pd.read_json('brevet_doi.json')

In [ ]:
df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

In [ ]:
df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

In [29]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54821 data in cached openalex data
6608 data in cached openalex data


In [30]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [31]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [32]:
current_dict = df_ipcc_ipbes_bso.to_dict(orient='records')

In [33]:
list_bso_ipcc_ipbes=[]
for i in range(len(df_ipcc_ipbes_bso)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [34]:
df_elastic_input=pd.DataFrame(current_dict)

In [35]:
df_elastic_input=df_ipcc_ipbes_bso

In [36]:
df_elastic_input=df_elastic_input[['doi', 'year', 'title', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes','topics']]

In [37]:
df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[None]),'authors_name']=None
df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[None]),'countries']=None
df_elastic_input.loc[df_elastic_input.institutions_names.apply(lambda x: x==[None]),'institutions_names']=None
df_elastic_input.loc[df_elastic_input.rors.apply(lambda x: x==[None]),'rors']=None
df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[]),'authors_name']=None
df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[]),'countries']=None
df_elastic_input.loc[df_elastic_input.institutions_names.apply(lambda x: x==[]),'institutions_names']=None
df_elastic_input.loc[df_elastic_input.rors.apply(lambda x: x==[]),'rors']=None
df_elastic_input.loc[df_elastic_input.year=='XXXX','year']=None

C:\Users\haallat\AppData\Local\Temp\ipykernel_5476\4250329376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[None]),'authors_name']=None
C:\Users\haallat\AppData\Local\Temp\ipykernel_5476\4250329376.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[None]),'countries']=None
C:\Users\haallat\AppData\Local\Temp\ipykernel_5476\4250329376.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

In [38]:
len(df_elastic_input)

58240

In [43]:
df_elastic_input=df_elastic_input.dropna(subset=['countries'])

In [58]:
df_elastic_input[df_elastic_input.countries.apply(lambda x: 'FRA' in x)].reset_index()#.doi.to_json('list_doi_fr.json',orient='records')

,index,doi,year,title,rors,ipcc,authors_name,institutions_names,countries,ipbes,topics
0,38,10.1002/2013gl057775,2013.0,{Global-scale seasonally resolved black carbon...,"[[https://ror.org/033tt8e33, US], [https://ror...","[{'name': 'wg1_chap_06', 'wg': '1', 'chap': 6}]","[[J. P. Schwarz, [US]], [B. H. Samset, [NO]], ...","[[NOAA Earth System Research Laboratory, US], ...","[GBR, FRA, USA, NOR]",NaN,"[Atmospheric Aerosols and their Impacts, Strat..."
1,44,10.1002/2013gl058118,2013.0,{Robust increase in equilibrium climate sensit...,"[[https://ror.org/05esem239, DE], [https://ror...","[{'name': 'wg1_chap_07', 'wg': '1', 'chap': 7}]","[[Katharina Meraner, [DE]], [Thorsten Mauritse...","[[Max Planck Institute for Meteorology, DE], [...","[DEU, FRA]",NaN,"[Climate Change and Variability Research, Glob..."
2,52,10.1002/2013gl058511,2013.0,{Modeling soil moisture–precipitation feedback...,"[[https://ror.org/00pggkr55, GB], [https://ror...","[{'name': 'wg1_chap_08', 'wg': '1', 'chap': 8}...","[[Christopher M. Taylor, [GB]], [Cathryn E. Bi...","[[UK Centre for Ecology & Hydrology, GB], [Uni...","[GBR, SWE, FRA]",NaN,"[Climate Change and Variability Research, Nume..."
3,54,10.1002/2013gl058653,2014.0,{Optimal fingerprinting under multiple sources...,"[[https://ror.org/05k07f122, AR], [https://ror...","[{'name': 'wg1_chap_03', 'wg': '1', 'chap': 3}]","[[Alexis Hannart, [AR]], [Aurélien Ribes, [FR]...",[[Instituto Franco-Argentino sobre Estudios de...,"[ARG, FRA]",NaN,"[Total Least Squares Methods and Applications,..."
4,70,10.1002/2013jc009071,2013.0,{Skill testing a three-dimensional global tide...,"[[https://ror.org/05g3dte14, US], [https://ror...","[{'name': 'wg1_chap_09', 'wg': '1', 'chap': 9}]","[[Patrick G. Timko, [GB, US]], [Brian K. Arbic...","[[Florida State University, US], [University o...","[GBR, FRA, USA]",NaN,"[Oceanic Modeling and Circulation Studies, Glo..."
...,...,...,...,...,...,...,...,...,...,...,...
4249,58055,10.5194/bg-7-2851-2010,2010,"Deep, diverse and definitely different: unique...","[[https://ror.org/01bpa4157, ES], [https://ror...",NaN,"[[Eva Ramirez-Llodra, [ES]], [Angelika Brandt,...","[[Institut Català de Ciències del Clima, ES], ...","[GBR, FRA, ESP, ITA, DEU, CAN, USA, NOR, BEL]",[{'chapter': '2.2_nature'}],[Marine Biodiversity and Ecosystem Functioning...
4250,58057,10.5194/bg-7-979-2010,2010,Projected 21st century decrease in marine prod...,"[[https://ror.org/02k7v4d05, CH], [https://ror...",NaN,"[[Marco Steinacher, [CH]], [F. Joos, [CH]], [T...","[[University of Bern, CH], [Oeschger Centre fo...","[CHE, DEU, USA, FRA]",[{'chapter': '4'}],[Marine Biogeochemistry and Ecosystem Dynamics...
4251,58064,10.5194/essd-5-165-2013,2013,The global carbon budget 1959–2011,"[[https://ror.org/040tfy969, GB], [https://ror...",NaN,"[[Corinne Le Quéré, [GB]], [R. J. Andres, [US]...","[[Tyndall Centre, GB], [University of East Ang...","[GBR, FRA, NLD, ITA, AUS, JPN, DEU, USA, NOR, ...",[{'chapter': '3'}],"[Global Methane Emissions and Impacts, Atmosph..."
4252,58230,10.7717/peerj.2249,2016,A horizon scan of future threats and opportuni...,"[[https://ror.org/04g2vpn86, GB], [https://ror...",NaN,"[[Mark J. F. Brown, [GB]], [Lynn V. Dicks, [GB...","[[Royal Holloway University of London, GB], [C...","[CHN, GBR, FRA, BRA, ZAF, AUS, IRL, DEU, USA, ...",[{'chapter': '6'}],"[Impact of Pesticides on Honey Bee Health, Imp..."


In [47]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [48]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

fichier excel

In [2]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [3]:
df=df[['doi','ipcc','ipbes']]

In [4]:
df

,doi,ipcc,ipbes
0,10.1001/jama.2012.108940,"[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6}]",NaN
1,10.1001/jama.2014.13186,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",NaN
2,10.1001/jama.2014.15715,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",NaN
3,10.1001/jama.2020.1097,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",NaN
4,10.1001/jama.2020.15398,"[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4}...",NaN
...,...,...,...
58235,10.7770/cuhso-v27n1-art1134,NaN,[{'chapter': '3'}]
58236,10.7930/j0x63jt0,NaN,[{'chapter': 'ipbes-global_glossary'}]
58237,10.9755/ejfa.v25i1.15376,NaN,[{'chapter': '3'}]
58238,10.9758/cpn.2015.13.3.239,NaN,[{'chapter': '2.3_ncp'}]


In [5]:
df[(pd.isna(df.ipcc)==False)&(pd.isna(df.ipbes)==False)]

,doi,ipcc,ipbes
137,10.1002/2014ef000238,"[{'name': 'wg3_chap_03', 'wg': '3', 'chap': 3}]",[{'chapter': '5'}]
153,10.1002/2014gb004949,"[{'name': 'wg1_chap_07', 'wg': '1', 'chap': 7}...",[{'chapter': '4'}]
154,10.1002/2014gb004995,"[{'name': 'wg1_chap_05', 'wg': '1', 'chap': 5}...",[{'chapter': '4'}]
189,10.1002/2014gl060382,"[{'name': 'wg1_chap_08', 'wg': '1', 'chap': 8}]",[{'chapter': '4'}]
635,10.1002/2015jg003036,"[{'name': 'wg2_chap_13', 'wg': '2', 'chap': 13...",[{'chapter': '4'}]
...,...,...,...
52846,10.5751/es-08388-210247,"[{'name': 'wg2_chap_18', 'wg': '2', 'chap': 18}]","[{'chapter': '4'}, {'chapter': '5'}]"
52851,10.5751/es-08685-210404,"[{'name': 'wg2_chap_17', 'wg': '2', 'chap': 17}]",[{'chapter': '5'}]
52857,10.5751/es-08804-210409,"[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4}...","[{'chapter': '2.3_ncp'}, {'chapter': '3'}]"
53076,10.7208/chicago/9780226145006.001.0001,"[{'name': 'wg2_chap_02', 'wg': '2', 'chap': 2}]","[{'chapter': '1'}, {'chapter': '2.1_drivers'},..."


In [ ]:
df.loc[:,'ipbes']=df.loc[:,'ipbes'].apply(lambda x: [y['chapter'] for y in x] if isinstance(x, list) else None)

In [ ]:
df.loc[:,'ipcc']=df.loc[:,'ipcc'].apply(lambda x: [y['name'] for y in x] if isinstance(x, list) else None)

In [8]:
df.drop_duplicates('doi')

,doi,ipcc,ipbes
0,10.1001/jama.2012.108940,[wg2_chap_06],[]
1,10.1001/jama.2014.13186,[wg2_chap_07],[]
2,10.1001/jama.2014.15715,[wg2_chap_07],[]
3,10.1001/jama.2020.1097,[wg2_chap_07],[]
4,10.1001/jama.2020.15398,"[wg2_chap_04, wg2_chap_07, wg2_cross_chap_2]",[]
...,...,...,...
58235,10.7770/cuhso-v27n1-art1134,[],[3]
58236,10.7930/j0x63jt0,[],[ipbes-global_glossary]
58237,10.9755/ejfa.v25i1.15376,[],[3]
58238,10.9758/cpn.2015.13.3.239,[],[2.3_ncp]


In [9]:
df.to_json('ipcc_ipbes_chapters.jsonl', orient= 'records', lines=True)

In [ ]:
df.loc[:,'rors']=df.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'authors_name']=df.loc[:,'authors_name'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'institutions_names']=df.loc[:,'institutions_names'].apply(lambda x: [[]] if x==[None] else x)

In [ ]:
df.loc[pd.isna(df['rors'])==False,'rors']=df.loc[pd.isna(df['rors'])==False,'rors'].apply(lambda x: ','.join([j.replace('https://ror.org/','') for j in list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())]))

In [ ]:
df.loc[pd.isna(df['institutions_names'])==False,'institutions_names']=df.loc[pd.isna(df['institutions_names'])==False,'institutions_names'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())))

In [ ]:
df.loc[pd.isna(df['authors_name'])==False,'authors_name']=df.loc[pd.isna(df['authors_name'])==False,'authors_name'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y.keys()) for y in x if isinstance(y,dict)])).drop_duplicates())))

In [ ]:
df.loc[pd.isna(df['ipcc'])==False,'ipcc']=df.loc[pd.isna(df['ipcc'])==False,'ipcc'].apply(lambda x: ','.join(list((pd.Series([y.get('name') for y in x]).drop_duplicates()))))

In [ ]:
df.loc[pd.isna(df['ipbes'])==False,'ipbes']=df.loc[pd.isna(df['ipbes'])==False,'ipbes'].apply(lambda x: ','.join(list((pd.Series([y.get('chapter') for y in x]).drop_duplicates()))))

In [ ]:
df['is_fr']=df['countries'].apply(lambda x: 'France' in x if isinstance(x,list) else False)

In [ ]:
df.loc[pd.isna(df['countries'])==False,'countries']=df.loc[pd.isna(df['countries'])==False,'countries'].apply(lambda x: ','.join(x))

In [ ]:
df[['doi','title','ipcc','ipbes','countries','is_fr','rors','institutions_names','authors_name']].to_excel('doi_ipcc_ipbes.xlsx', index=False)